In [1]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments


Import Log

In [2]:
log = xes_importer.apply("noisy_log.xes")

C:\Users\I518079\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 9135.35it/s]


In [3]:

bpmn_graph = pm4py.read_bpmn("../diagram.bpmn")
net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

Find Deviations

In [4]:
found_allignments = alignments.apply(log, net, im, fm)

aligning log, completed variants :: 100%|██████████| 20/20 [00:00<00:00, 434.42it/s]


Draw Sankey

In [5]:
import pandas as pd
trace_attribute_values = [trace.attributes['Bank'] for trace in log]
attribute_count = {}
for attribute in trace_attribute_values:
    if attribute in attribute_count:
        attribute_count[attribute] += 1
    else:
        attribute_count[attribute] = 1
attributes = sorted(attribute_count.keys())
labels = sorted(set(trace_attribute_values))
deviations = ['No Skip Deviation','Skip Deviation']
sankey_value = {}

for i, found_allignment in enumerate(found_allignments):
    
    if found_allignment['cost'] > 1000:
        if log[i].attributes['Bank'] not  in sankey_value:
            sankey_value[log[i].attributes['Bank']] = [0,1]
        else:
            sankey_value[log[i].attributes['Bank']] [1] += 1
    else:
        if log[i].attributes['Bank'] not  in sankey_value:
            sankey_value[log[i].attributes['Bank']]  = [1,0]
        else:
            sankey_value[log[i].attributes['Bank']][0] += 1
source = []
target = []
value = []
deviation_count = 0
no_deviation_count = 0
for key in sankey_value:
    positions = labels.index(key)
    source.append(positions)
    source.append(positions)
    target.append(3)
    target.append(4)
    value.append(sankey_value[key][0])
    no_deviation_count += sankey_value[key][0]
    value.append(sankey_value[key][1])
    deviation_count += sankey_value[key][1]
    
labels = [f"{attr} ({attribute_count[attr]})" for attr in attributes] + deviations
labels[3] = f"No Skip Deviation ({no_deviation_count})"
labels[4] = f"Skip Deviation ({deviation_count})"
    

In [6]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = labels,
      color = "blue"
    ),
    link = dict(
      source = source, 
      target = target,
      value = value
  ))])

fig.update_layout(title_text="Deviations based on Trace Attribute", font_size=10)
fig.update_layout(width=int(400))
fig.show()
